<a href="https://colab.research.google.com/github/kungfumas/aplikasi-deep-learning/blob/master/pysparkbaru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark on Google Colab 101

In this article, we will see how we can run PySpark in a Google Colaboratory notebook. We will also perform some basic data exploratory tasks common to most data science problems. So, let’s get cracking!

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [6]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz

In [7]:
!tar xf /content/spark-3.2.2-bin-hadoop3.2.tgz

In [8]:
!pip install -q findspark

In [9]:
!ls -a

.   .config	 spark-3.2.2-bin-hadoop3.2
..  sample_data  spark-3.2.2-bin-hadoop3.2.tgz


In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

In [13]:
import findspark
findspark.init()

In [14]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [15]:
spark

In [16]:
!wget --continue https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json -O /tmp/sample_books.json

--2022-08-31 06:00:32--  https://raw.githubusercontent.com/GarvitArya/pyspark-demo/main/sample_books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘/tmp/sample_books.json’

/tmp/sample_books.j 100%[===================>]   1.53K  --.-KB/s    in 0s      

2022-08-31 06:00:32 (20.4 MB/s) - ‘/tmp/sample_books.json’ saved [1565/1565]



In [17]:
 
# Read JSON file into dataframe
df = spark.read.json("/tmp/sample_books.json")


In [18]:
df.printSchema()


root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [19]:
df.show(20,False)

+--------------------+-----------------+-----+-----------------------------+------------+
|author              |edition          |price|title                        |year_written|
+--------------------+-----------------+-----+-----------------------------+------------+
|Austen, Jane        |Penguin          |18.2 |Northanger Abbey             |1814        |
|Tolstoy, Leo        |Penguin          |12.7 |War and Peace                |1865        |
|Tolstoy, Leo        |Penguin          |13.5 |Anna Karenina                |1875        |
|Woolf, Virginia     |Harcourt Brace   |25.0 |Mrs. Dalloway                |1925        |
|Cunnningham, Michael|Harcourt Brace   |12.35|The Hours                    |1999        |
|Twain, Mark         |Penguin          |5.76 |Huckleberry Finn             |1865        |
|Dickens, Charles    |Random House     |5.75 |Bleak House                  |1870        |
|Twain, Mark         |Random House     |7.75 |Tom Sawyer                   |1862        |
|Woolf, Vi

In [20]:
df.count()

13

In [21]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



In [22]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")

In [23]:
df_filtered.select("title", "price", "year_written").show(5, False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



In [24]:
df_filtered.count()

3

In [25]:
df_filtered.show(20, False)

+--------------------+-----------------+-----+-----------------------------+------------+
|author              |edition          |price|title                        |year_written|
+--------------------+-----------------+-----+-----------------------------+------------+
|Cunnningham, Michael|Harcourt Brace   |12.35|The Hours                    |1999        |
|Rowling, J.K.       |Harcourt Brace   |19.95|Harry Potter                 |2000        |
|Marquez             |Harper  Perennial|14.0 |One Hundred Years of Solitude|1967        |
+--------------------+-----------------+-----+-----------------------------+------------+



In [26]:
df_filtered.select("title", "year_written").filter("title LIKE '%Harry Potter%'").distinct().show(20, False)


+------------+------------+
|title       |year_written|
+------------+------------+
|Harry Potter|2000        |
+------------+------------+



In [27]:
from pyspark.sql.functions import max
# Find the costliest book

maxValue = df_filtered.agg(max("price")).collect()[0][0]
print("maxValue: ",maxValue)
df_filtered.select("title","price").filter(df.price == maxValue).show(20, False)

maxValue:  19.95
+------------+-----+
|title       |price|
+------------+-----+
|Harry Potter|19.95|
+------------+-----+

